# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [41]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [42]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [43]:
# Load API Credentials

with open("/Users/Benjamin/.secret/yelp_api.json", 'r') as f:
    login = json.load(f)

login.keys()

dict_keys(['client-id', 'api-key'])

In [44]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

### Define Search Terms and File Paths

In [45]:
# set our API call parameters and filename before the first call
search_location = 'Bothell, WA'
search_term = 'Lasagna'

In [46]:
## Specify fodler for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER + f"{search_location.split(',')[0]}_{search_term}.json".lower()
JSON_FILE

'Data/bothell_lasagna.json'

In [47]:
results = yelp_api.search_query(location=search_location, term= search_term, timeout_s = 5)
type(results)

dict

In [48]:
results['businesses']

[{'id': '4DAuRaxv5VskRbvOglv-fQ',
  'alias': 'grazie-ristorante-italiano-bothell',
  'name': 'Grazie Ristorante Italiano',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/15RNj6j0Ph4QSL2R0V6fQA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/grazie-ristorante-italiano-bothell?adjust_creative=cP2n0TumDRZwvea-GVoqOg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=cP2n0TumDRZwvea-GVoqOg',
  'review_count': 460,
  'categories': [{'alias': 'italian', 'title': 'Italian'},
   {'alias': 'bars', 'title': 'Bars'}],
  'rating': 4.0,
  'coordinates': {'latitude': 47.7870237893, 'longitude': -122.21883907633},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '23207 Bothell Everett Hwy',
   'address2': '',
   'address3': '',
   'city': 'Bothell',
   'zip_code': '98021',
   'country': 'US',
   'state': 'WA',
   'display_address': ['23207 Bothell Everett Hwy', 'Bothell, WA 98021']},
  'phone': '+14254029600',
  'displa

In [49]:
def nested_dict_outline(dict_object, indent=5):
    indent = indent + 10
    if type(dict_object) == dict:
        for key in dict_object:
            print(f"\n{key:>{indent}} {type(dict_object[key])}", end="")
            nested_dict_outline(dict_object[key],indent)
    elif type(dict_object) == list:
         print(f" with length {len(dict_object)}",end="")
    elif type(dict_object) == int:
         print(f" with value {dict_object}",end="")
              
nested_dict_outline(results)


     businesses <class 'list'> with length 20
          total <class 'int'> with value 158
         region <class 'dict'>
                   center <class 'dict'>
                          longitude <class 'float'>
                           latitude <class 'float'>

### Check if Json File exists and Create it if it doesn't

In [50]:
## Check if JSON_FILE exists

if os.path.isfile(JSON_FILE) == False:
## If it does not exist: 
    ## inform user and save empty list
    print("The file does not exist. Creating empty file")
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump(results['businesses'], f)
        
## If it exists, inform user
else:
    print("File alread exists.")

The file does not exist. Creating empty file


### Load JSON FIle and account for previous results

In [51]:
## Load previous results and use len of results for offset

## set offset based on previous results


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [52]:
# use our yelp_api variable's search_query method to perform our API call


In [53]:
## How many results total?


- Where is the actual data we want to save?

In [54]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [55]:
# Use math.ceil to round up for the total number of pages of results.
import math
n_pages = math.ceil(results['total']/results_per_page)
n_pages

8

In [56]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
        
        
    ## Read in results in progress file and check the length
    with open(JSON_FILE) as f:
        prev_results = json.load(f)

    print(type(prev_results))
    n_results = len(prev_results)
        
    ## save number of results for to use as offset
    results = yelp_api.search_query(location = search_location,
                                    term = search_term,
                                    offset = n_results)    
    ## append new results and save to file
    prev_results.extend( results['businesses'] )    

    with open(JSON_FILE, 'w') as f:
        json.dump(prev_results, f)
    
    ## What to do if we get an error/exception.
        


  0%|          | 0/8 [00:00<?, ?it/s]

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [57]:
prev_results

[{'id': '4DAuRaxv5VskRbvOglv-fQ',
  'alias': 'grazie-ristorante-italiano-bothell',
  'name': 'Grazie Ristorante Italiano',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/15RNj6j0Ph4QSL2R0V6fQA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/grazie-ristorante-italiano-bothell?adjust_creative=cP2n0TumDRZwvea-GVoqOg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=cP2n0TumDRZwvea-GVoqOg',
  'review_count': 460,
  'categories': [{'alias': 'italian', 'title': 'Italian'},
   {'alias': 'bars', 'title': 'Bars'}],
  'rating': 4.0,
  'coordinates': {'latitude': 47.7870237893, 'longitude': -122.21883907633},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '23207 Bothell Everett Hwy',
   'address2': '',
   'address3': '',
   'city': 'Bothell',
   'zip_code': '98021',
   'country': 'US',
   'state': 'WA',
   'display_address': ['23207 Bothell Everett Hwy', 'Bothell, WA 98021']},
  'phone': '+14254029600',
  'displa

## Open the Final JSON File with Pandas

In [ ]:
# df = None

In [ ]:
# ## convert the filename to a .csv.gz
# csv_file = JSON_FILE.replace('.json','.csv.gz')
# csv_file

In [ ]:
## Save it as a compressed csv (to save space)


## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
# size_json = os.path.getsize(JSON_FILE)
# size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

# print(f'JSON FILE: {size_json:,} Bytes')
# print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

# print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

## Next Class: Processing the Results and Mapping 